<h2>Market Study - Chicken Exports</h2>

<a href="https://openclassrooms.com/en/" >Openclassrooms</a>, Data Analyst Course
<br>Project 5 - Michael Orange


<hr>

A chicken producer is looking for new export markets. All countries are considered. 
The core objective is to assist the company in targeting the markets with the highest potential. 

'Grouping' the countries in 'clusters' with similar caracteristics is expected. Variables are: 
- country population, 
- population growth, 
- food diet: calory supply per capita, protein supply per capita, 
- proportion of animal protein in the mix. 

Additional relevant variable might be included (ex. GDP per capita)

<hr>

<h2>Preparation Dataset data_country</h2>

<h3>Import Datasets</h3>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

Data are collected from The Food and Agriculture Organization Corporate Statistical Database (FAOSTAT). The FAOSTAT website disseminates statistical data collected and maintained by the Food and Agriculture Organization (FAO).

FAOSTAT - Data collected (2013 and 2017 are selected for the below analysis): analysis is based on 2017 figures, with a comparaison from 2013 data. 
2017 is the most recent comprehensive data from FAOSTAT.

- Datasets Food Balance Animal and Vegetal
- Dataset Population
- Dataset Gross Domestic Product (GDP)
- Dataset Trade Mix - Chicken 

In [2]:
#import food datasets
veg2013 = pd.read_csv("data/raw/vegetal2013.csv")
ani2013 = pd.read_csv("data/raw/animal2013.csv")
veg2017 = pd.read_csv('data/raw/vegetal2017.csv', dtype={'Note': str })
ani2017 = pd.read_csv("data/raw/animal2017.csv")
ani2013["origin"] = "animal"
veg2013["origin"] = "vegetal"
ani2017["origin"] = "animal"
veg2017["origin"] = "vegetal"

#import population
pop = pd.read_csv("data/raw/FAOSTAT_data_Population_2013-2017.csv")

# import GDP
gdp = pd.read_csv("data/raw/FAOSTAT_data_MacroIndicators_20132017.csv")

# import chicken trade
chicken_trade = pd.read_csv("data/raw/FAOSTAT_data_Trade_Chicken_20132017.csv")

# import chicken livestock
chicken_stock = pd.read_csv("data/raw/FAOSTAT_data_chicken_livestock_20132017.csv")

<h3>Preparation food dataset</h3>


In [3]:
# group food datatsets
temp = [veg2013, ani2013, ani2017, veg2017]
temp = pd.concat(temp, ignore_index=True)

# delete ani2013, veg2013, ani2017, veg2017
del ani2013, veg2013, ani2017, veg2017

temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element', 'item_code', 'item',
                'xx4', "year", "unit", "value", 'xx5', 'flag_description', 'origin', 'xx6']

data = temp.pivot_table(index=["year", "country_code", "country", "origin", "item_code", \
                               "item"], columns = ["element"], values=["value"], aggfunc=sum)

# rename columns
data.columns = ['domestic_supply_quantity','export_quantity','fat_supply_quantity_gcapitaday','feed',
                'food','food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr','import_quantity','losses','other_uses','processing',
                'production', 'protein_supply_quantity_gcapitaday', 'residuals', 'seed','stock_variation','tourist_consumption']

data = data.reset_index()

Data for China are duplicated. 
<br>data for 'China' = sum of the splitted data for 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan'.
- deletion of 'China' data (code 351)
- 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan' data are kept in order to preserve a better granularity.



In [4]:
# remove doublon from China (code_country 351)
data = data.loc[data.country_code != 351]

# remove Bermuda and Brunei - no information for 2017
data = data.loc[data.country != 'Bermuda']
data = data.loc[data.country != 'Brunei Darussalam']

<h3>Population data</h3>

- Population per country
- Growth of the population (%) - 2017 vs 2013

In [5]:
# population
pop.columns = ["xx","xx2","country_code","country", 'xx3', 'xx4', 'xx5','xx6', 
               'xx7', 'year',"xx8","population_total_thousand","xx9", 'xx10', 'xx11']

pop = pop.drop(["xx","xx2",'xx3','xx4','xx5', 'xx6', 'xx7','xx8', 'xx9', 'xx10', 'xx11'], axis = 1)
pop.reset_index(drop=True, inplace=True)

# merge data and pop
data = pd.merge(data, pop, how='left')

data_country = pop.pivot_table(index=['country_code', 'country'], columns = ['year'], values=['population_total_thousand'])
data_country.columns = ['pop_2013_thousand', 'pop_2017_thousand']
data_country.reset_index(inplace=True)

#growth population
data_country['pop_growth_percent'] = (data_country['pop_2017_thousand'] / data_country['pop_2013_thousand'] - 1) *100

<h3>Diet caracterisitics : proteins and calories </h3>

- Food supply kcal per capita per day - per country
- Protein supply g per capita per day - per country
- Growth of the protein supply per capita per country (%) - 2017 vs 2013

In [6]:
temp = data.pivot_table(index=['country_code', 'country'], columns=['year'], \
                        values=['food_supply_kcalcapitaday',  'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['food_supply_kcalcapitaday_2013', 'food_supply_kcalcapitaday_2017', \
                'protein_supply_gcapitaday_2013', 'protein_supply_gcapitaday_2017']
temp = temp.reset_index()

temp['protein_supply_capita_growth_percent'] = (temp['protein_supply_gcapitaday_2017'] / temp['protein_supply_gcapitaday_2013'] - 1) * 100

data_country = pd.merge(temp, data_country, how='left')

- Food supply kcal per country (kcal)
- Protein supply per country (kg)
- Growth of the protein supply per country (%) - 2017 vs 2013

In [7]:
NB_DAYS_YEAR = 365 

data_country['food_supply_kcal_2013'] = data_country['food_supply_kcalcapitaday_2013'] * NB_DAYS_YEAR * data_country['pop_2013_thousand'] * 1000
data_country['food_supply_kcal_2017'] = data_country['food_supply_kcalcapitaday_2017'] * NB_DAYS_YEAR * data_country['pop_2017_thousand'] * 1000

data_country['protein_supply_kg_2013'] = data_country['protein_supply_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000
data_country['protein_supply_kg_2017'] = data_country['protein_supply_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

data_country['protein_supply_growth_percent'] = (data_country['protein_supply_kg_2017'] /data_country['protein_supply_kg_2013'] - 1) * 100

Animal proteins per capita

- Protein supply g per capita per day from animals - per country
- Growth of the protein supply per capita per country (%) from animals - 2017 vs 2013


In [8]:
# protein per capita from animals
temp = data.loc[data['origin']=='animal'].pivot_table(index=['country_code', 'country'], columns = ['year'], \
                                                      values=[ 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['protein_supply_animal_gcapitaday_2013', 'protein_supply_animal_gcapitaday_2017']
temp.reset_index(inplace=True)

temp['protein_supply_capita_animal_growth_percent'] = \
    (temp['protein_supply_animal_gcapitaday_2017'] / temp['protein_supply_animal_gcapitaday_2013'] - 1) * 100

data_country = pd.merge(data_country, temp, how='left')


Animal proteins total

- Protein supply kg from animals - per country
- Growth of the protein supply per country (%) from animals - 2017 vs 2013

In [9]:
# protein supply from animals
data_country['protein_supply_animal_kg_2013'] = \
    data_country['protein_supply_animal_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000

data_country['protein_supply_animal_kg_2017'] = \
    data_country['protein_supply_animal_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

data_country['protein_supply_animal_growth_percent'] = (data_country['protein_supply_animal_kg_2017'] /data_country['protein_supply_animal_kg_2013'] - 1) * 100

Prevalence of animal proteins in the diet mix - per country

In [10]:
# prevalence of proteins from animals

data_country['prevalence_protein_animal_capita_percent_2013'] = \
    data_country['protein_supply_animal_gcapitaday_2013'] / data_country['protein_supply_gcapitaday_2013'] * 100
data_country['prevalence_protein_animal_capita_percent_2017'] = \
    data_country['protein_supply_animal_gcapitaday_2017'] / data_country['protein_supply_gcapitaday_2017'] * 100

data_country['prevalence_protein_animal_percent_2013'] = data_country['protein_supply_animal_kg_2013'] / data_country['protein_supply_kg_2013'] * 100
data_country['prevalence_protein_animal_percent_2017'] = data_country['protein_supply_animal_kg_2017'] / data_country['protein_supply_kg_2017'] * 100

In [11]:
# growth of the prevalence of proteins from animals
data_country['prevalence_protein_animal_capita_growth_percent'] = \
    (data_country['prevalence_protein_animal_capita_percent_2017'] /data_country['prevalence_protein_animal_capita_percent_2013'] - 1) * 100

data_country['prevalence_protein_animal_growth_percent'] = \
    (data_country['prevalence_protein_animal_percent_2017'] /data_country['prevalence_protein_animal_percent_2013'] - 1) * 100

<h3>Gross Domestic Product (GDP) per capita</h3>

- GDP per capita (USD)
- GDP Growth - 2017 vs 2013

In [12]:
gdp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "year", "xx7", "value", 'xx8', 'xx9', 'xx10']
gdp = gdp.loc[gdp['item'] == 'Gross Domestic Product per capita'].pivot_table(index=["year", "country_code", "country"], columns = ['item'], values=['value'])
gdp.columns = ['gdp_percapita_usd']

gdp = gdp.pivot_table(index=["country_code", "country"], columns = ['year'], values=['gdp_percapita_usd'])
gdp.columns = ['gdp_percapita_usd_2013', 'gdp_percapita_usd_2017']
gdp.reset_index(inplace=True)

data_country = pd.merge(data_country, gdp, how='left')

GDP of Taiwan is missing. 
Informaiton collected from the International Monetary Fund (IMF).
- GDP per capita 2013 : 43 739 USD
- GDP per capita 2017 : 50 593 USD

Source: https://bit.ly/2U2251b

In [13]:
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2013'] = 43739
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2017'] = 50593

In [14]:
#growth of the GDP
gdp['gdp_percapita_growth'] = (gdp['gdp_percapita_usd_2017'] / gdp['gdp_percapita_usd_2013'] - 1) * 100

<h3>Poultry</h3>



Poultry are domesticated avian species that can be raised for eggs, meat and/or feathers. The term “poultry” covers a wide range of birds, from indigenous and commercial breeds of chickens to Muscovy ducks, mallard ducks, turkeys, guinea fowl, geese, quail, pigeons, ostriches and pheasants. 

- In 2017, chickens accounted for some 92 percent of the world’s poultry population, followed by ducks (5 percent), and turkeys (2 percent). 
- Chickens contribute 89 percent of world poultry meat production, followed by turkeys with 5 percent, ducks with 4 percent and geese and guinea fowl with 2 percent. The rest comes from other poultry species.
- Chickens provide 92 percent of world egg production.

source: http://www.fao.org/poultry-production-products/production/en/

There is no granular data about chicken exclusively in the dataset 'animal' but the data for the category 'Poultry' (in which chicken is accounting around 90% of the total) give us a fair indicator of the chicken market in each country. 

Poultry composition: Meat chicken, Fat liver prepared (foie gras), Meat chicken canned, Meat,duck, Meat goose and guinea fowl, Meat turkey.

In [15]:
temp = data.loc[(data['item'] == 'Poultry Meat')|(data['item'] == 'Eggs')].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=['domestic_supply_quantity', 'food', 'import_quantity', 'protein_supply_quantity_gcapitaday'], aggfunc=sum)

temp.columns = ['domestic_supply_poultry_tons_2013', 'domestic_supply_poultry_tons_2017', 'food_poultry_tons_2013',  'food_poultry_tons_2017',\
                'import_poultry_tons_2013', 'import_poultry_tons_2017', 'protein_supply_poultry_gcapitaday_2013', 'protein_supply_poultry_gcapitaday_2017']

temp.reset_index(inplace=True)
temp

#convert 1000 tons to tons
temp['domestic_supply_poultry_tons_2013'] = temp['domestic_supply_poultry_tons_2013'] * 1000
temp['domestic_supply_poultry_tons_2017'] = temp['domestic_supply_poultry_tons_2017'] * 1000
temp['food_poultry_tons_2013'] = temp['food_poultry_tons_2013'] * 1000
temp['food_poultry_tons_2017'] = temp['food_poultry_tons_2017'] * 1000
temp['import_poultry_tons_2013'] = temp['import_poultry_tons_2013'] * 1000
temp['import_poultry_tons_2017'] = temp['import_poultry_tons_2017'] * 1000

data_country = pd.merge(data_country, temp, how='left')

In [16]:
#growth Poultry+Eggs protein supply
data_country['food_poultry_growth'] = (data_country['food_poultry_tons_2017'] / data_country['food_poultry_tons_2013'] - 1) * 100
#growth Poultry+Eggs imports
data_country['import_poultry_growth'] = (data_country['import_poultry_tons_2017'] / data_country['import_poultry_tons_2013'] - 1) * 100

<h3>Chicken Importations</h3>

Data are from the food and agricultural trade dataset collected, processed and disseminated by FAO. The data is mainly provided by UNSD, Eurostat, and other national authorities as needed.
- Import quantity of 'chicken' per country
- Import value of 'chicken' per country

The category 'chicken' is made of 'chicken', 'meat, chicken', 'meat, chicken, canned', 'offals, liver chicken'. 
Offals liver chicken was not available in the 'Poultry' section.  

In [17]:
chicken_trade.columns =['xx', 'xx2', 'country_code', 'country', 'partner_country_code', 'partner_country', 'xx6', \
                        'element', 'xx8', 'xx9', 'xx10', 'year', 'unit', 'value', 'xx13', 'xx14', 'xx15']

# there is a disruptancy on the name of the UK between the FAO trade dataset and other FAO datasets
chicken_trade.loc[chicken_trade['country'] == 'United Kingdom', 'country'] = 'United Kingdom of Great Britain and Northern Ireland'
chicken_trade.loc[chicken_trade['partner_country'] == 'United Kingdom', 'partner_country'] = 'United Kingdom of Great Britain and Northern Ireland'

In [18]:
#Imports of chicken per country - in quantity and value

chicken_import = chicken_trade.loc[((chicken_trade['element'] == 'Import Value') | (chicken_trade['element'] == 'Import Quantity')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

chicken_import = chicken_import.pivot_table(index=["country_code", "country"], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_import.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons_2017', \
                         'import_chicken_value_thousand_usd_2017']
chicken_import.reset_index(inplace=True)

data_country = pd.merge(data_country, chicken_import, how='left')

In [19]:
country_no_report_import_chicken = \
    data_country.loc[(data_country['import_chicken_quantity_tons_2017'].isna()) | (data_country['import_chicken_quantity_tons_2013'].isna())]

{{ len(country_no_report_import_chicken)}} countries do have missing information about chicken importations due to the absence of reports in 2013 or 2017:

{{country_no_report_import_chicken.country.tolist()}}

We will get an approximation of these imports through the declaration of exports from partner countries.

In [20]:
# getting information about the chicken imports on the countries not reporting the imports through the export reports from other countries.

chicken_export = chicken_trade.loc[((chicken_trade['element'] == 'Export Quantity') | (chicken_trade['element'] == 'Export Value')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

# Export of chicken to countries not reporting any chicken imports
chicken_export_to_countries_noreport = chicken_export.loc[chicken_export['partner_country'].isin(country_no_report_import_chicken.country)]

chicken_export_to_countries_noreport = chicken_export_to_countries_noreport.pivot_table(\
                                                                                       index=['partner_country_code', 'partner_country'], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_export_to_countries_noreport.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', \
                                                'import_chicken_quantity_tons_2017', 'import_chicken_value_thousand_usd_2017']
chicken_export_to_countries_noreport.reset_index(inplace=True)

chicken_export_to_countries_noreport.rename(columns = {'partner_country_code':'country_code', 'partner_country':'country'}, inplace=True)

In [21]:
data_country.set_index('country', inplace=True)
chicken_export_to_countries_noreport.set_index('country', inplace=True)

#overlapping values from chicken export to data_country for the countries not reporting their imports
data_country = data_country.combine_first(chicken_export_to_countries_noreport)
data_country.reset_index(inplace=True)

In [22]:
data_country['import_chicken_value_growth_percent'] = \
    (data_country['import_chicken_value_thousand_usd_2017'] / data_country['import_chicken_value_thousand_usd_2013'] - 1) *100

data_country['import_chicken_quantity_growth_percent'] = \
    (data_country['import_chicken_quantity_tons_2017'] / data_country['import_chicken_quantity_tons_2013'] - 1) *100

data_country['import_chicken_ratio_usd_tons'] = \
    data_country['import_chicken_value_thousand_usd_2017'] * 1000 / data_country['import_chicken_quantity_tons_2013']

<h3>Chicken Livestock Patterns</h3>

The Livestock Patterns contains (among others) data on livestock numbers, shares of major livestock species. 

- Units for the stock is Livestock Units (LSU)
- Units for the share in total livestock is % of the total LSU 


In [23]:
chicken_stock.columns =['xx', 'xx1', 'country_code', 'country', 'xx3', 'element', 'xx4', 'xx5', 'xx6', 'year', 'xx7', 'value', 'xx8', 'xx9']

chicken_stock = chicken_stock.loc[chicken_stock['element'] != 'Livestock units per agricultural land area']

chicken_stock = chicken_stock.pivot_table(index=["country_code", "country"], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_stock.columns = ['share_chicken_total_units_2013', 'livestock_chicken_units_2013', 'share_chicken_total_units_2017', 'livestock_chicken_units_2017']
chicken_stock.reset_index(inplace=True)

data_country = pd.merge(data_country, chicken_stock, how='left')

In [24]:
data_country['share_chicken_growth_percent'] = \
    (data_country['share_chicken_total_units_2017'] / data_country['share_chicken_total_units_2013'] - 1) *100

data_country['livestock_chicken_growth_percent'] = \
    (data_country['livestock_chicken_units_2017'] / data_country['livestock_chicken_units_2013'] - 1) *100

Analysis is made on 2017 figures and growth 2013-2017, removing columns with 2013 information

In [28]:
data_country = data_country.drop([ 'pop_2013_thousand', 'food_supply_kcal_2013',
       'food_supply_kcal_2017', 'food_supply_kcalcapitaday_2013',
       'food_supply_kcalcapitaday_2017', 'gdp_percapita_usd_2013',
       'gdp_percapita_usd_2017', 'import_chicken_quantity_tons_2013',
       'import_chicken_value_thousand_usd_2013',
       'prevalence_protein_animal_capita_percent_2013',
       'prevalence_protein_animal_percent_2013',
       'protein_supply_animal_gcapitaday_2013',
       'protein_supply_animal_growth_percent', 'protein_supply_animal_kg_2013',
       'protein_supply_gcapitaday_2013',
       'protein_supply_kg_2013', 'import_poultry_tons_2013', 'food_poultry_tons_2013', 
        'food_poultry_tons_2013', 'protein_supply_poultry_gcapitaday_2013',  'domestic_supply_poultry_tons_2013', 
        'share_chicken_total_units_2013', 'livestock_chicken_units_2013'], axis = 1)

In [29]:
data_country

,country,country_code,domestic_supply_poultry_tons_2017,food_poultry_growth,food_poultry_tons_2017,import_chicken_quantity_tons_2017,import_chicken_value_thousand_usd_2017,import_poultry_growth,import_poultry_tons_2017,pop_2017_thousand,...,protein_supply_growth_percent,protein_supply_kg_2017,protein_supply_poultry_gcapitaday_2017,import_chicken_value_growth_percent,import_chicken_quantity_growth_percent,import_chicken_ratio_usd_tons,share_chicken_total_units_2017,livestock_chicken_units_2017,share_chicken_growth_percent,livestock_chicken_growth_percent
0,Afghanistan,2,131000.0,-5.217391,109000.0,70001.0,86403.0,7.792208,83000.0,36296.113,...,4.427092,7.165887e+08,0.98,47.081454,64.086637,2025.339303,2.04,135730.00,13.966480,12.610968
1,Albania,3,97000.0,8.536585,89000.0,40034.0,52618.0,52.000000,38000.0,2884.169,...,6.574962,1.258002e+08,10.58,21.427088,66.669442,2190.591174,10.54,78350.00,-11.651299,-12.242384
2,Algeria,4,667000.0,0.988468,613000.0,1991.0,25354.0,-33.333333,2000.0,41389.189,...,9.616670,1.402690e+09,4.37,-6.210927,-26.774549,9324.751747,20.69,1314355.20,-2.174941,1.104246
3,Angola,7,347000.0,-15.404040,335000.0,267388.0,271266.0,-18.032787,300000.0,29816.766,...,8.246401,5.886679e+08,3.79,-38.967973,-22.429454,786.955727,8.89,365000.00,21.282401,34.156468
4,Antigua and Barbuda,8,7000.0,0.000000,5000.0,6455.0,12779.0,16.666667,7000.0,95.426,...,1.350000,2.826494e+06,18.56,2.240179,9.965928,2177.001704,14.54,1579.23,4.454023,5.282000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Venezuela (Bolivarian Republic of),236,790000.0,-48.571429,702000.0,25033.0,61682.0,-91.103203,25000.0,29402.484,...,-23.815305,6.042063e+08,8.21,-91.569178,-91.089620,219.554214,8.37,1205946.84,1.948843,-1.151898
168,Viet Nam,237,1740000.0,6.464646,1581000.0,363681.0,393775.0,-41.000000,295000.0,94600.648,...,14.994997,3.108667e+09,5.39,-33.404533,-30.725185,750.071907,19.00,2952093.66,15.431349,25.884024
169,Yemen,249,312000.0,-11.392405,280000.0,77952.0,121190.0,-28.440367,78000.0,27834.819,...,0.051381,5.409029e+08,3.50,-51.958868,-28.257989,1115.354881,14.88,650000.00,4.347826,3.174603
170,Zambia,251,111000.0,3.125000,99000.0,12070.0,11117.0,500.000000,12000.0,16853.599,...,20.762037,3.633113e+08,1.84,143.900834,669.279796,7085.404716,13.55,403338.90,2.963526,6.141816
